# Directory

> directory class.

In [ ]:
#| default_exp utils.directory

In [ ]:
#| hide
from nbdev.showdoc import *

In [17]:
#| export
import os
from pathlib import Path
from dataclasses import dataclass, field, KW_ONLY
from typing import Optional, List, ClassVar

### Directory Viewer

#### Directory

In [ ]:
#| export 
@dataclass
class Directory:
    dirname: str

    _: KW_ONLY
    _SPACE : ClassVar[str] = '    '
    _BRANCH: ClassVar[str] = '│   '    
    _TEE   : ClassVar[str] = '├── '
    _LAST  : ClassVar[str] = '└── '

        
    def make_tree(self, dirname:Path, prefix:str=''):
        '''
        A recursive generator, given a directory Path object
        will yield a visual tree structure line by line
        with each line prefixed by the same characters
        Notes
        -----
        Adapted from https://stackoverflow.com/a/59109706/5623899
        '''

        contents = list(dirname.iterdir())
        # NOTE: contents each get pointers that are ├── with a final └── :
        pointers = [self._TEE] * (len(contents) - 1) + [self._LAST]
        for pointer, path in zip(pointers, contents):
            yield f'{prefix}{pointer}{path.name}'

            # NOTE: extend the prefix and recurse:
            if path.is_dir(): 
                # NOTE: space because last, └── , above so no more |
                extension = self._BRANCH if pointer == self._TEE else self._SPACE 
                
                yield from self.make_tree(path, prefix=f'{prefix}{extension}')

    def get_tree_lines(self, dirname:Optional[str]=None) -> List[str]:
        dirname = self.prepare_dirname(dirname)
        tree = self.make_tree(dirname)
        lines = [line for line in tree]
        return lines

    def make_tree_str(self, dirname:Optional[str]=None) -> str:
        dirname = self.prepare_dirname(dirname)
        lines = self.get_tree_lines(dirname)
        tree_str = '\n'.join([str(dirname), *lines])
        return tree_str
        
    def prepare_dirname(self, dirname:Optional[str]=None) -> Path:
        if dirname is None:
            dirname = self.dirname

        dirname = os.path.expanduser(dirname) 
        dirname = os.path.abspath(dirname) 

        if not isinstance(dirname, Path):
            dirname = Path(dirname)

        return dirname

    def print(self, dirname:Optional[str]=None) -> None:
        dirname = self.prepare_dirname(dirname)
        tree_str = self.make_tree_str(dirname)
        print(tree_str)
        return

    def __repr__(self):
        dirname = self.dirname
        dirname = self.prepare_dirname(dirname)
        tree_str = self.make_tree_str(dirname)        
        return tree_str        

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()